# Copy of KNN_Confusions.ipynb 
Except this copy is written for the CIFAR10 dataset

In [ ]:
###############
## Libraries ##
###############

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import keras

# Load Data

In [ ]:
######################################
## Import CIFAR10 data from Scratch ##
######################################
import pickle

# unpickle the binary files
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# paths to each batch of data
batch1 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_1")
batch2 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_2")
batch3 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_3")
batch4 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_4")
batch5 = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/data_batch_5")
meta = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/batches.meta")
test = unpickle("/scratch/gpfs/eysu/src_data/cifar-10-batches-py/test_batch")

# separate labels and image data from each batch
y_train1 = batch1[b'labels']
x_train1 = batch1[b'data']
y_train2 = batch2[b'labels']
x_train2 = batch2[b'data']
y_train3 = batch3[b'labels']
x_train3 = batch3[b'data']
y_train4 = batch4[b'labels']
x_train4 = batch4[b'data']
y_train5 = batch5[b'labels']
x_train5 = batch5[b'data']

# concatenate into big training and testing arrays
y_train = np.concatenate((y_train1, y_train2, y_train3, y_train4, y_train5))
x_train = np.concatenate((x_train1, x_train2, x_train3, x_train4, x_train5), axis=0)


# shuffle the training data identically to before
def shuffle_in_unison(x, y, permutation):
    assert x.shape[0] == y.shape[0]
    shuffled_x = np.empty(x.shape, dtype=x.dtype)
    shuffled_y = np.empty(y.shape, dtype=y.dtype)
    for old_index, new_index in enumerate(permutation):
        shuffled_x[new_index] = x[old_index]
        shuffled_y[new_index] = y[old_index]
        
    return shuffled_x, shuffled_y
        
permutation = np.loadtxt('/scratch/gpfs/eysu/src_data/cifar-10-batches-py/permutation.csv', delimiter=',').astype(np.int64)

x_train, y_train = shuffle_in_unison(x_train, y_train, permutation)

y_test = test[b'labels']
x_test = test[b'data']

In [ ]:
#################################################
## Preprocess data by reshaping and separating ##
#################################################
labels = ['airplane',  # index 0
          'automobile',  # index 1
          'bird',  # index 2 
          'cat',  # index 3 
          'deer',  # index 4
          'dog',  # index 5
          'frog',  # index 6 
          'horse',  # index 7 
          'ship',  # index 8 
          'truck']  # index 9

# Further break training data into train / validation sets 
# put 5000 into validation set and keep remaining 45,000 for train
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# reshape data to match dimensions of cifar10.load_data
x_train = x_train.reshape(45000, 3, 32, 32)
x_train = x_train.transpose(0, 2, 3, 1)
x_train = x_train.astype('float32')
x_train /= 255

# y_train = tf.keras.utils.to_categorical(y_train, 10)

x_valid = x_valid.reshape(5000, 3, 32, 32)
x_valid = x_valid.transpose(0, 2, 3, 1)
x_valid = x_valid.astype('float32')
x_valid /= 255
y_valid = tf.keras.utils.to_categorical(y_valid, 10)

x_test = x_test.reshape(10000, 3, 32, 32)
x_test = x_test.transpose(0, 2, 3, 1)
x_test = x_test.astype('float32')
x_test /= 255
y_test = tf.keras.utils.to_categorical(y_test, 10)

# assert dimensions of data
print("TRAINING DATA")
print(x_train.shape)
print(y_train.shape)

print("VALIDATION DATA")
print(x_valid.shape)
print(y_valid.shape)

print("TESTING DATA")
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Examine any image

# Image index, you can pick any number between 0 and 44,999
img_index = 10
label_index = y_train[img_index]
# Print the label, for example 2 Pullover
print("y = " + str(label_index) + " (" +(labels[label_index]) + ")")
plt.imshow(x_train[img_index])
plt.show()

In [ ]:
################################################
## Recreate model from final iterations of SR ##
################################################

# specify path of model to load in
model_weights = '/scratch/gpfs/eysu/CIFAR10_results/og_training/model.weights.best.iter24.hdf5'

# load model
model = load_model(model_weights)

# examine model
model.summary()
model.get_weights()
model.optimizer

layer_output = model.layers[-2].output
activation_model = models.Model(inputs=model.input, outputs=layer_output)
print(activation_model)

# Run image through model and get activations
activations = activation_model.predict(x_train) # should be 2 numpy arrays of dimension N images x 256D

print(activations.shape)

In [ ]:
############################################################################################
## Read prediction softmax layer activations for all train set images, and all iterations ##
############################################################################################

MAX_ITER = 25

# Build arrays of dimensions: N training images X L labels X P iterations
y_hat_train_arr = np.zeros([y_train.shape[0], len(labels), MAX_ITER])

for i in range(MAX_ITER):
    if i == 0:
        y_hat_train_name = 'y_hat_train_seed'
    else:
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)
        
    # Load test set softmax outputs 
    yhtr = np.load('/scratch/gpfs/eysu/CIFAR10_results/og_training/' + y_hat_train_name + '.npy')

    # The first time through, use binary weight vectors to save correct class array
    if i == 0:
        true_class_tr = np.nonzero(yhtr)[1]
        
    y_hat_train_arr[:, :, i] = yhtr
    
final_predictions = np.argmax(y_hat_train_arr[:,:, 24], axis=1)

print(y_hat_train_arr.shape)
print(final_predictions.shape)

# If 1st time, find K Nearest Neighbors

In [ ]:
############################
## Find Nearest Neighbors ##
############################

NUM_IMAGES = x_train.shape[0]
NUM_NEIGHBORS = 4

# create arrays to store results

nearest_neighbors_in_class = np.zeros([NUM_IMAGES, NUM_NEIGHBORS])
nearest_neighbors_in_class_norms = np.zeros([NUM_IMAGES, NUM_NEIGHBORS])
nearest_neighbors_other_class = np.zeros([NUM_IMAGES, NUM_NEIGHBORS])
nearest_neighbors_other_class_norms = np.zeros([NUM_IMAGES, NUM_NEIGHBORS])

for image_idx in range(NUM_IMAGES):
    # determine image class and divide indices by in/out of class
    image_class = y_train[image_idx]
    same_class_idxs = np.array(np.where(y_train == image_class)).squeeze()
    other_class_idxs = np.array(np.where(y_train != image_class)).squeeze()

    # find norm of differences between original image and every other image
    diffs = activations - activations[image_idx]
    norms = np.linalg.norm(diffs, axis=1)

    # get nearest neighbors in and out of class
    min_in_class_idxs = np.argpartition(norms[same_class_idxs], range(4))
    min_other_class_idxs = np.argpartition(norms[other_class_idxs], range(4))
    
    # this is a bit confusing but since np.argpartition returns the indices of the min
    # values in the subarray that we pass it, we need to use the in_class_idxs and other_class_idxs
    # arrays to convert these indices back to the main array indices
    min_in_class_idxs = same_class_idxs[min_in_class_idxs]
    min_other_class_idxs = other_class_idxs[min_other_class_idxs]
    
    # update all results arrays
    # start with the 2nd index since the nearest neighbor is always itself
    
    nearest_neighbors_in_class[image_idx, :] = min_in_class_idxs[1:5]
    nearest_neighbors_in_class_norms[image_idx, :] = norms[min_in_class_idxs[1:5]]
    
    nearest_neighbors_other_class[image_idx, :] = min_other_class_idxs[1:5]
    nearest_neighbors_other_class_norms[image_idx, :] = norms[min_other_class_idxs[1:5]]
    
    print(str(np.around(image_idx/NUM_IMAGES, 2)))
    
nearest_neighbors_in_class = nearest_neighbors_in_class.astype(int)
nearest_neighbors_other_class = nearest_neighbors_other_class.astype(int)
    
print("done")

In [ ]:
# export nearest neighbors of all images
import pandas as pd
pd.DataFrame(nearest_neighbors_in_class).to_csv("Outputs/NearestNeighbors/nearest_neighbors_in_class.csv", sep = ',', header=None, index=None)
pd.DataFrame(nearest_neighbors_in_class_norms).to_csv("Outputs/NearestNeighbors/nearest_neighbors_in_class_norms.csv", sep = ',', header=None, index=None)

pd.DataFrame(nearest_neighbors_other_class).to_csv("Outputs/NearestNeighbors/nearest_neighbors_other_class.csv", sep = ',', header=None, index=None)
pd.DataFrame(nearest_neighbors_other_class_norms).to_csv("Outputs/NearestNeighbors/nearest_neighbors_other_class_norms.csv", sep = ',', header=None, index=None)


# Else, load K Nearest Neighbors

In [ ]:
############################
## Load Nearest Neighbors ##
############################

# code to import nearest neighbors matrices
import pandas as pd
# read in 45000 x 25 matrices of 25 nearest in class neighbors and their norms

# digits
nearest_neighbors_in_class = pd.read_csv("Outputs/NearestNeighbors/nearest_neighbors_in_class.csv", sep = ',', header=None).to_numpy()
nearest_neighbors_in_class_norms = pd.read_csv("Outputs/NearestNeighbors/nearest_neighbors_in_class_norms.csv", sep = ',', header=None).to_numpy()

nearest_neighbors_other_class = pd.read_csv("Outputs/NearestNeighbors/nearest_neighbors_other_class.csv", sep = ',', header=None).to_numpy()
nearest_neighbors_other_class_norms = pd.read_csv("Outputs/NearestNeighbors/nearest_neighbors_other_class_norms.csv", sep = ',', header=None).to_numpy()

print(nearest_neighbors_in_class.shape)
print(nearest_neighbors_in_class)

In [ ]:
####################################################
## Helper Function Make 9XN plots of 4 nearest in ##
## and out of class neighbors for given image     ##
####################################################


def helper_plot_4NN(image_idx):
    # plot image in center with 4 least confusing in and out of class image alongside
    figure = plt.figure(figsize=(20, 8), constrained_layout=True)
    spec = figure.add_gridspec(ncols=9, nrows=1)

    ax1 = figure.add_subplot(spec[0,0], xticks=[], yticks=[])
    im1 = ax1.imshow(x_train[nearest_neighbors_in_class[image_idx, 3]], cmap='gray')
    ax1.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 3]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 3], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 3]]]), color = 'green')

    ax2 = figure.add_subplot(spec[0,1], xticks=[], yticks=[])
    im2 = ax2.imshow(x_train[nearest_neighbors_in_class[image_idx, 2]], cmap='gray')
    ax2.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 2]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 2], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 2]]]), color = 'green')
    
    ax3 = figure.add_subplot(spec[0,2], xticks=[], yticks=[])
    im3 = ax3.imshow(x_train[nearest_neighbors_in_class[image_idx, 1]], cmap='gray')
    ax3.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 1]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 1], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 1]]]), color = 'green')

    ax4 = figure.add_subplot(spec[0,3], xticks=[], yticks=[])
    im4 = ax4.imshow(x_train[nearest_neighbors_in_class[image_idx, 0]], cmap='gray')
    ax4.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 0]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 0], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 0]]]), color = 'green')

    ax5 = figure.add_subplot(spec[0,4], xticks=[], yticks=[])
    im5 = ax5.imshow(x_train[image_idx], cmap='gray')
    ax5.set_title("Class: " + str(labels[y_train[image_idx]]) + "\nImage" + 
                  "\nFinal predicted class: " + str(labels[final_predictions[image_idx]]), color = 'green')


    ax6 = figure.add_subplot(spec[0,5], xticks=[], yticks=[])
    im6 = ax6.imshow(x_train[nearest_neighbors_other_class[image_idx, 0]], cmap='gray')
    ax6.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 0]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 0], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 0]]]), color = 'red')
    
    ax7 = figure.add_subplot(spec[0,6], xticks=[], yticks=[])
    im7 = ax7.imshow(x_train[nearest_neighbors_other_class[image_idx, 1]], cmap='gray')
    ax7.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 1]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 1], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 1]]]), color = 'red')
  
                  
    ax8 = figure.add_subplot(spec[0,7], xticks=[], yticks=[])
    im8 = ax8.imshow(x_train[nearest_neighbors_other_class[image_idx, 2]], cmap='gray')
    ax8.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 2]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 2], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 2]]]), color = 'red')
   

    ax9 = figure.add_subplot(spec[0,8], xticks=[], yticks=[])
    im9 = ax9.imshow(x_train[nearest_neighbors_other_class[image_idx, 3]], cmap='gray')
    ax9.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 3]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 3], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 3]]]), color = 'red')

    plt.show()
    return figure
    
    

In [ ]:
###################################
## Helper Function for 4xN plots ##
###################################

def print_images_helper(index, softmax, image, title):
    figure = plt.figure(figsize=(40, 40))
    # plot image
    ax1 = figure.add_subplot(8, 8, 1, xticks=[], yticks=[])
    im1 = ax1.imshow(image, cmap='gray')
    ax1.set_title(title)

    # plot weights graph
    if (softmax[:, 0].all() == softmax[:, 1].all()):
        title_str = "Softmax Outputs (iter 1 same)"
        
    else: title_str = "Softmax Outputs"
        
    ax2 = figure.add_subplot(8, 8, 2)
    im2 = ax2.imshow(softmax.T, cmap='Wistia')
    
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = figure.colorbar(im2, cax=cax, ticks=[0, 1])
    cbar.ax.set_yticklabels(['0', '1'])

    ax2.set(xlabel='Classes', ylabel='Iterations', title=title_str)
    ax2.set_xticks(ticks=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    ax2.set_xticklabels(labels, rotation = 'vertical')

    # plot correlation graph
    corr_arr = np.corrcoef(softmax.T)

    ax3 = figure.add_subplot(8, 8, 3)
    im3 = ax3.imshow(corr_arr, cmap='Wistia')
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])
    cbar.ax.set_yticklabels(['0', '1'])
    ax3.set_title("Correlation Matrix")

    # plot eigenvalues graph
    eigs, _ = np.linalg.eig(corr_arr)
    num_nonzero = np.count_nonzero(np.around(eigs, 2))

    if (num_nonzero == 1):
        title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzero)"
    else:
        title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzeros)"

    ax4 = figure.add_subplot(8, 8, 4)
    im4 = ax4.plot(eigs, marker='o')
    ax4.set(xlabel="PC Number", xlim=[0,len(eigs)], ylim=[0,MAX_ITER], title=title_str)

    plt.show()
    return figure
    

In [ ]:
########################
## Visualize N images ##
########################

from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings('ignore')
import matplotlib.backends.backend_pdf

def visualize_N_images(CLASS, NUM_IMAGES, rank):
    print("Class: ", CLASS)
    # least confusing
    
    pdf = matplotlib.backends.backend_pdf.PdfPages("Outputs/NearestNeighbors/least_confusing_" + str(labels[CLASS]) + "s.pdf")
    print(str(NUM_IMAGES) + " least confusing " + str(labels[CLASS]) + "s")
    for i in range(NUM_IMAGES):
        # get index of image
        image_idx = rank[i]
        
        # print image itself
        figure = print_images_helper(image_idx, y_hat_train_arr[image_idx], x_train[image_idx], "Original Image")
        pdf.savefig(figure, bbox_inches='tight')

        figure2 = helper_plot_4NN(image_idx)
        pdf.savefig(figure2, bbox_inches='tight')
        
    pdf.close()

    # most confusing
    
    pdf = matplotlib.backends.backend_pdf.PdfPages("Outputs/NearestNeighbors/most_confusing_" + str(labels[CLASS]) + "s.pdf")
    print(str(NUM_IMAGES) + " most confusing " + str(labels[CLASS]) + "s")
    for i in range(NUM_IMAGES):
        # get index of image
        image_idx = rank[-NUM_IMAGES + i]
        
        # print image itself
        figure = print_images_helper(image_idx, y_hat_train_arr[image_idx], x_train[image_idx], "Original Image")
        pdf.savefig(figure, bbox_inches='tight')

        figure2 = helper_plot_4NN(image_idx)
        pdf.savefig(figure2, bbox_inches='tight')
        
    pdf.close()

In [ ]:
##############################################
## Rank by difference to binary seed vector ##
##############################################

# Call visualization methods and generate PDFs from here

for i in range(10):
    # choose which class to analyze
    CLASS = i
    same_class_idxs = np.array(np.where(y_train == CLASS)).squeeze()
    class_data = y_hat_train_arr[np.where(true_class_tr == CLASS), :, :].squeeze()
    diff_arr = class_data[:, CLASS, 0] - class_data[:, CLASS, MAX_ITER - 1]
    seed_diff_rank = np.argsort(diff_arr)
    
    # again, convert subarray indices back to master indices
    seed_diff_rank = same_class_idxs[seed_diff_rank]
    print(seed_diff_rank)
    visualize_N_images(i, 50, seed_diff_rank)